In [6]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
from collections import Counter
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
#Load Dataframe
data = Path('Loan_Predictions/Loans_Data_ML.csv')
df = pd.read_csv(data)
df.head()

,Loan_Amt,Loan_Term,Monthly_Payment,Credit_Grade,Employement_Length,Annual_Income,DTI_Ratio,Application_Type,Joint_Annual_Income,Joint_DTI_Ratio,...,Delinquent_Amount,#_Mortgage_Accounts,#_Bankruptcies,#_Tax_Liens,Balance_Exc_Mortgage,Joint_Revolving_Balance,Open_Acc_IL24m,Open_RevAcc_24m,Acc_Curr_PD30days,Loan_Status
0,3600,36,111.97,A4,10+ years,120000.0,18.90,INDIVIDUAL,0.0,0.0,...,0,0,1,0,36506,0.0,14,11,0.0,Issued
1,15000,60,356.78,C4,10+ years,125000.0,17.25,INDIVIDUAL,0.0,0.0,...,0,2,0,0,69364,0.0,3,3,0.0,Issued
2,8400,36,276.56,B3,8 years,50000.0,15.63,INDIVIDUAL,0.0,0.0,...,0,4,0,0,51591,0.0,5,12,0.0,Issued
3,4000,36,130.00,B2,2 years,50000.0,33.61,INDIVIDUAL,0.0,0.0,...,0,6,0,0,136208,0.0,7,4,0.0,Issued
4,6000,36,185.93,A3,3 years,125000.0,9.25,INDIVIDUAL,0.0,0.0,...,0,0,0,0,60622,0.0,15,3,0.0,Current


In [12]:
# Remove the `Issued` loan status
issued_mask = df['Loan_Status'] != 'Issued'
df = df.loc[issued_mask]



# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,Loan_Amt,Loan_Term,Monthly_Payment,Credit_Grade,Employement_Length,Annual_Income,DTI_Ratio,Application_Type,Joint_Annual_Income,Joint_DTI_Ratio,...,Delinquent_Amount,#_Mortgage_Accounts,#_Bankruptcies,#_Tax_Liens,Balance_Exc_Mortgage,Joint_Revolving_Balance,Open_Acc_IL24m,Open_RevAcc_24m,Acc_Curr_PD30days,Loan_Status
0,6000,36,185.93,A3,3 years,125000.0,9.25,INDIVIDUAL,0.0,0.0,...,0,0,0,0,60622,0.0,15,3,0.0,low_risk
1,12000,36,370.48,A2,10+ years,56000.0,17.90,INDIVIDUAL,0.0,0.0,...,0,5,0,0,21540,0.0,5,0,0.0,low_risk
2,5000,36,173.31,C4,3 years,60000.0,13.32,INDIVIDUAL,0.0,0.0,...,0,2,0,0,25092,0.0,3,1,0.0,Fully Paid
3,10500,60,345.15,G2,< 1 year,45000.0,17.25,INDIVIDUAL,0.0,0.0,...,0,0,1,0,22148,0.0,7,13,0.0,low_risk
4,18000,36,542.07,A1,< 1 year,155000.0,9.08,INDIVIDUAL,0.0,0.0,...,0,0,0,0,61257,0.0,4,1,0.0,low_risk


## Split the Data into Training and Testing

In [14]:
# Create our features

X = df.drop(columns='Loan_Status', axis=1)
X = pd.get_dummies(X)

# Create our target
y = df["Loan_Status"]

In [15]:
X.describe()

,Loan_Amt,Loan_Term,Monthly_Payment,Annual_Income,DTI_Ratio,Joint_Annual_Income,Joint_DTI_Ratio,#_Open_Accounts,#_Delinquent_Accounts,Total_Amount_in_Collection,...,Employement_Length_5 years,Employement_Length_6 years,Employement_Length_7 years,Employement_Length_8 years,Employement_Length_9 years,Employement_Length_< 1 year,Employement_Length_None,Application_Type_DIRECT_PAY,Application_Type_INDIVIDUAL,Application_Type_JOINT
count,81708.000000,81708.000000,81708.000000,8.170800e+04,81708.000000,8.170800e+04,81708.000000,81708.000000,81708.000000,81708.000000,...,81708.000000,81708.000000,81708.000000,81708.000000,81708.000000,81708.000000,81708.000000,81708.000000,81708.000000,81708.000000
mean,14873.249253,42.033485,452.014293,8.277918e+04,21.630156,5.025092e+03,0.766943,11.805282,0.006535,252.506144,...,0.062479,0.043900,0.033497,0.033326,0.036961,0.071144,0.069859,0.000269,0.956797,0.042933
std,9380.311900,10.411629,283.724738,2.278298e+05,172.179865,2.651908e+04,3.887579,5.744327,0.085587,3747.889257,...,0.242024,0.204874,0.179932,0.179488,0.188667,0.257066,0.254910,0.016407,0.203314,0.202708
min,1000.000000,36.000000,30.120000,0.000000e+00,-1.000000,0.000000e+00,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7950.000000,36.000000,246.880000,4.920000e+04,12.270000,0.000000e+00,0.000000,8.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,12000.000000,36.000000,373.220000,7.000000e+04,18.010000,0.000000e+00,0.000000,11.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,20000.000000,60.000000,604.240000,1.000000e+05,24.360000,0.000000e+00,0.000000,15.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,40000.000000,60.000000,1715.420000,6.100000e+07,9999.000000,1.210000e+06,61.900000,71.000000,3.000000,932461.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# Check the balance of our target values
y.value_counts()

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(61281, 72)

### Oversampling

## Naive Random Oversampling

In [19]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)


ModuleNotFoundError: No module named 'imblearn'

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
# Naive Random Oversampling
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

## SMOTE Oversampling

In [ ]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
# SMOTE Oversampling
print(classification_report_imbalanced(y_test, y_pred))

## Undersampling

In [ ]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=78)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
# Undersampling
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

## Combination (Over and Under) Sampling


In [ ]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
# Combination (Over and Under) Sampling
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))